In [38]:
import time
from openai import OpenAI
client = OpenAI()

In [39]:
# file upload
batch_input_file = client.files.create(
  file=open("../data/ch02_batchinput.jsonl", "rb"),
  purpose="batch"
)

# Create a new batch
client.batches.create(
    input_file_id=batch_input_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "테스트를 위한 배치",
    }
)
start_time = time.time()

In [ ]:
# Batch 객체들의 리스트를 가져옵니다.
batches = client.batches.list().data

# 각 배치의 상세 정보를 출력합니다.
for batch in batches[:5]:
    print(f"Batch ID: {batch.id}")
    print(f"Status: {batch.status}")
    print(f"Created At: {batch.created_at}")
    print(f"input_file_id: {batch.input_file_id}")
    print(f"output_file_id: {batch.output_file_id}")
    print("------")

Batch ID: batch_674338f7552881908768f70dae8e5ea9
Status: validating
Created At: 1732458743
input_file_id: file-Q8mYsoemnm5DxZzjiPHUZY
output_file_id: None
------
Batch ID: batch_674336d52b688190b0d397478747745f
Status: completed
Created At: 1732458197
input_file_id: file-Cj8Lt9CWeyuJZ925C2ktKF
output_file_id: file-7ztiejaK3fWFWvfVBxZqUg
------
Batch ID: batch_674335e4d4ac81909430e1e4f05dc8a3
Status: completed
Created At: 1732457956
input_file_id: file-L88QzYT6zQbzhDtsH8GCJa
output_file_id: file-T2RKu1GkBaRZvsbmKPZMDM
------
Batch ID: batch_6742fb58f76c8190b8241508f9b54704
Status: completed
Created At: 1732442968
input_file_id: file-FHYHLzZog8azWz7UH8e917
output_file_id: file-EA5oLpKAtv5vA2SEnSdnhX
------
Batch ID: batch_6742f89082b48190a2d8c3c039e531db
Status: completed
Created At: 1732442256
input_file_id: file-46zaS3KEVdwSD4m55PAa1M
output_file_id: file-RFNg3WjW4UVw1VFRYLEBsg
------
Batch ID: batch_6742f7ac98cc8190bd35110ec2e0e955
Status: completed
Created At: 1732442028
input_file_i

In [ ]:
# status를 15초에 한번씩 확인하고 'completed'일때까지 로그를 출력
# status가 'completed'가 되면 루프를 탈출합니다.
# 총 걸린 시간을 추가로 출력합니다.
while True:
  status = client.batches.retrieve(client.batches.list().data[0].id).status
  print("time: ", time.strftime('%X', time.localtime()), "   |    status: ", status)
  if status == "completed":
    break
  time.sleep(60) # 60초에 한번씩 확인

print("Total time: ", time.time() - start_time)

# output file을 다운로드 받아서 출력
file_response = client.files.content(client.batches.list().data[0].output_file_id)
print(file_response.text.encode('utf-8').decode('unicode-escape'))

In [ ]:
# # https://raisonde.tistory.com/entry/한글-↔-유니코드-간편-변환기
# 유니코드 : \uccab\ubc88\uc9f8 \uc131\uacf5!
# 한글 : 첫번째 성공!

In [ ]:
file_response.text.encode('utf-8').decode('unicode-escape').split("\n")[0].split(",")[10]

' "content": "[첫번째 성공!]"'

In [ ]:
# "../data/ch02_batchinput.jsonl", "rb" 해당경로에 저장
with open("../data/ch03_batchinput_sentencePair_세종실록_filtered_sampled_10.jsonl", "w", encoding="utf-8") as f:
    for request in file_response:
        f.write(file_response.dumps(request, ensure_ascii=False) + "\n")